<a href="https://colab.research.google.com/github/ashishagg70/GANText2Image/blob/master/Text2FaceV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! wget http://www.cs.toronto.edu/~rkiros/models/dictionary.txt
# ! wget http://www.cs.toronto.edu/~rkiros/models/utable.npy
# ! wget http://www.cs.toronto.edu/~rkiros/models/btable.npy
# ! wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz
# ! wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz.pkl
# ! wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz
# ! wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz.pkl

In [1]:
!gdown --id 1D4yFM5bLF2PEVlM_yXzTp-cLjGgNai2m
#!gdown --id 1UQNd2OsbuuW5QSfc6-78e0heJk3F3FeL
#!gdown --id 14a90ceynctKczgase-iVbS8ZbwcTZSOT
#!gdown --id 1d1Cfsz1BcdWE_b4eF8KG6tSa0efsKLZ0
!gdown --id 19DTv9tXIby7FW8TnU69u8xbNpcTWx6RG

Downloading...
From: https://drive.google.com/uc?id=1D4yFM5bLF2PEVlM_yXzTp-cLjGgNai2m
To: /content/img_align_celeba.zip
1.44GB [00:20, 70.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=19DTv9tXIby7FW8TnU69u8xbNpcTWx6RG
To: /content/list_attr_celeba.csv.zip
2.12MB [00:00, 68.0MB/s]


In [2]:
!gdown --id 1VMBG_cAfbUZ7CJmzuQhkuPkSPBtkGOti #skipthoughts.py module for import ready python3.7
!gdown --id 1Z2nZ0sykKPn64BvzqrbwUjRFC4VqJAhm #annotated data Text2FaceAnnotation.json
!gdown --id 1SYFQOQ1LSlGCgsr3QnzxCN-LDuUjEnYv #skip thought vectors of dim 4500 of descriptions
!gdown --id 1KajlSVNK_UfPLnv5d-gNlwCSxcmkk4XO #filename in order of skip thought loaded

Downloading...
From: https://drive.google.com/uc?id=1VMBG_cAfbUZ7CJmzuQhkuPkSPBtkGOti
To: /content/skipthoughts.py
100% 13.9k/13.9k [00:00<00:00, 13.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Z2nZ0sykKPn64BvzqrbwUjRFC4VqJAhm
To: /content/Text2FaceAnnotation.json
100% 1.46M/1.46M [00:00<00:00, 94.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SYFQOQ1LSlGCgsr3QnzxCN-LDuUjEnYv
To: /content/image_data_skip_thought_vectors_of_descriptions.npy
109MB [00:03, 34.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KajlSVNK_UfPLnv5d-gNlwCSxcmkk4XO
To: /content/images_filename_in_order_of_skip_thought_vectors.txt
100% 62.5k/62.5k [00:00<00:00, 50.7MB/s]


In [3]:
!unzip img_align_celeba.zip
!unzip list_attr_celeba.csv.zip
#!unzip list_bbox_celeba.csv.zip
#!unzip list_eval_partition.csv.zip
#!unzip list_landmarks_align_celeba.csv.zip

Streaming output truncated to the last 5000 lines.
 extracting: img_align_celeba/197602.jpg  
 extracting: img_align_celeba/197603.jpg  
 extracting: img_align_celeba/197604.jpg  
 extracting: img_align_celeba/197605.jpg  
 extracting: img_align_celeba/197606.jpg  
 extracting: img_align_celeba/197607.jpg  
 extracting: img_align_celeba/197608.jpg  
 extracting: img_align_celeba/197609.jpg  
 extracting: img_align_celeba/197610.jpg  
 extracting: img_align_celeba/197611.jpg  
 extracting: img_align_celeba/197612.jpg  
 extracting: img_align_celeba/197613.jpg  
 extracting: img_align_celeba/197614.jpg  
 extracting: img_align_celeba/197615.jpg  
 extracting: img_align_celeba/197616.jpg  
 extracting: img_align_celeba/197617.jpg  
 extracting: img_align_celeba/197618.jpg  
 extracting: img_align_celeba/197619.jpg  
 extracting: img_align_celeba/197620.jpg  
 extracting: img_align_celeba/197621.jpg  
 extracting: img_align_celeba/197622.jpg  
 extracting: img_align_celeba/197623.jpg  
 ex

In [4]:
import numpy as np
import torch
import sys
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
from nltk.tokenize import word_tokenize
from torchvision import datasets, transforms
from PIL import Image
from torchvision.transforms import ToPILImage
import torchvision.utils as vutil
import json
from collections import defaultdict
import nltk
import skipthoughts
from importlib import reload 
import tqdm 
reload(skipthoughts)
nltk.download('punkt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
cuda


In [6]:
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)
skip_thought_vector=encoder.encode([string_to_encode])

Loading model parameters...
Compiling encoders...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Loading tables...
Packing up...


In [5]:
skip_thought_filename='image_data_skip_thought_vectors_of_descriptions.npy'
order_filename='images_filename_in_order_of_skip_thought_vectors.txt'
json_filename='Text2FaceAnnotation.json'

In [6]:
order_filename_list = open(order_filename, "r").read().splitlines()

with open(skip_thought_filename, 'rb') as f:
    skip_thought_list = np.load(f)

assert len(order_filename_list)==skip_thought_list.shape[0]

In [7]:
#description
#filename
#response_id
#user_id
#max=381 and min =38 description lengths
file=open(json_filename,'r')
data=json.load(file)
data_dict={}
data_list=[]
for i,jsonElement in enumerate(data):
  data_dict[jsonElement['filename']]=jsonElement['description']
  data_list.append((jsonElement['filename'],jsonElement['description']))

In [24]:
# list_attr_celeba_test = pd.read_csv("list_attr_celeba.csv")
# #list_attr_celeba_test.head(10)
# attributes_test = list_attr_celeba_test.iloc[:,1:].values
# attributes_test[attributes_test<0]=0
# #attributes_test=np.tile(attributes_test,6)

In [8]:
image_dir='./img_align_celeba/'
inp_features = 4800
features=256
z_features = 100
imagePixel=64

In [9]:
transform = transforms.Compose([transforms.Resize(imagePixel),
                                transforms.CenterCrop(imagePixel),
                                transforms.ToTensor(),transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])])

In [25]:
images_attr_data = []
i=0
for filename, description_skip_thought in zip(order_filename_list, skip_thought_list):
  image = Image.open(image_dir+filename).convert('RGB')
  x = transform(image).numpy()
  i=i+1
  images_attr_data.append((x,description_skip_thought))

In [26]:
def collate_fn(data):
  imgs = list(map(lambda i: images_attr_data[i][0],data))
  attrs= list(map(lambda i: images_attr_data[i][1],data))
  l = len(data)
  
  imgs = torch.tensor(imgs).to(device)
  attrs = torch.tensor(attrs)
  attrs = attrs.to(device)
  return imgs,attrs

In [27]:
train_loader = torch.utils.data.DataLoader(
    range(len(images_attr_data)-1),
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)


In [28]:
def weights_init(m):
    if type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [29]:
class nn_G(nn.Module):
    def __init__(self):
      super().__init__()
      self.no_of_conv_layers=np.log2(imagePixel//4).astype('int32')
      dim = (imagePixel//8)*64
      flat_dim = 4*4*dim

      self.conv=[]
      self.bnorm=[]
      self.linear0 = nn.Linear(inp_features,features)
      self.linear1 = nn.Linear(features+z_features,flat_dim)

      for i in range(self.no_of_conv_layers-1):
        new_dim = dim//2
        self.conv.append(nn.ConvTranspose2d(dim,new_dim,4,stride=2, padding=1))
        self.bnorm.append( nn.BatchNorm2d(new_dim))
        dim = new_dim
      self.conv=nn.ModuleList(self.conv)
      self.bnorm=nn.ModuleList(self.bnorm)
      self.conv_final = nn.ConvTranspose2d(64,3,4,stride=2, padding=1)   
      

    def forward(self,x,z):
      x=self.linear0(x)
      x = torch.hstack((z,x))
      #print("gen start",x.shape)
      x = self.linear1(x)
      #print("lin",x.shape)

      l = len(x[0])
      x = x.reshape((len(x),l//(4*4),4,4))
      #print("reshape",x.shape)

     
      for i in range(self.no_of_conv_layers-1):
        x = self.conv[i](x)
        x = self.bnorm[i](x)
        x = nn.LeakyReLU(0.2)(x)
        #print("conv",i,x.shape)
 
      x = self.conv_final(x)
      #print("conv_final",x.shape)
      
      x = nn.Tanh()(x)
      #print("tanh",x.shape)

      return x


In [30]:
class nn_D(nn.Module):
    def __init__(self):
      super().__init__()

      self.no_of_conv_layers=np.log2(imagePixel//4).astype('int32')
      #print('no-of-conv-layer init',self.no_of_conv_layers)
      in_channel=3
      out_channel=64
      dim = (imagePixel//4)*64
      self.conv=[]
      self.bnorm=[]

      for i in range(self.no_of_conv_layers):
        self.conv.append(nn.Conv2d(in_channel,out_channel,4,stride=2, padding=1))
        self.bnorm.append(nn.BatchNorm2d(out_channel))
        in_channel = out_channel
        out_channel = out_channel*2
      self.conv=nn.ModuleList(self.conv)
      self.bnorm=nn.ModuleList(self.bnorm)

     
      self.lineart = nn.Linear(inp_features,features*4*4)
      self.conv5 = nn.Conv2d(in_channel+features,1,4)
      self.linear = nn.Linear((in_channel+features)*4*4,1)

    def forward(self,x,t):
      #print("dis start",x.shape, t.shape)
      #print('no-of-conv-layer',self.no_of_conv_layers)

      for i in range(self.no_of_conv_layers):
        x = self.conv[i](x)
        x = self.bnorm[i](x)
        x = nn.LeakyReLU(0.2)(x)
        #print("conv",i,x.shape)

      t1=self.lineart(t)
      t = t1.reshape((t.shape[0],features,4,4))
      #print("stack t",t.shape)

      #print(x.shape, t.shape)
      x = torch.cat((x,t),dim=1)
      x = self.conv5(x)
      #print("conv5",x.shape)

      x=x.squeeze()
      x = nn.Sigmoid()(x)
      #print("final",x.shape)

      return x


In [35]:
nn_Gen = nn_G().to(device)
nn_Dis = nn_D().to(device)
nn_Gen.apply(weights_init)
nn_Dis.apply(weights_init)

nn_D(
  (conv): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  )
  (bnorm): ModuleList(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lineart): Linear(in_features=4800, out_features=4096, bias=True)
  (conv5): Conv2d(768, 1, kernel_size=(4, 4), stride=(1, 1))
  (linear): Linear(in_features=12288, out_features=1, bias=True)
)

In [36]:

epochs = 100

learning_rate_dis = 0.0001
learning_rate_gen=0.0001*2

optimizer_gen = torch.optim.Adam(nn_Gen.parameters(),lr=learning_rate_gen,betas=(0.5,0.5))

optimizer_dis = torch.optim.Adam(nn_Dis.parameters(),lr=learning_rate_dis,betas=(0.5,0.5))

criterion = nn.BCELoss(reduction='mean')
#criterion=wasserstein_loss



In [ ]:

for epoch in range(epochs):
    for batch_idx, _data in enumerate(tqdm.notebook.tqdm(iter(train_loader))):
      _imgs, _attrs= _data
      optimizer_dis.zero_grad()
      out_real = nn_Dis(_imgs,_attrs)
      l = len(_attrs)
      real_labels = (1.1 - 0.9) * torch.rand((l,)) + 0.9
      real_tensor = torch.tensor(real_labels,dtype=torch.float).to(device)
      fake_labels = (0.1 - 0.0) * torch.rand((l,)) + 0.0
      fake_tensor = torch.tensor(fake_labels,dtype=torch.float).to(device)

      if(batch_idx%3!=0):
        loss_real = criterion(out_real,real_tensor)
      else:
        loss_real = criterion(out_real,fake_tensor)
      loss_real.backward()
      z= torch.randn((l,z_features))
      z=z.to(device)
      out_fake = nn_Dis(nn_Gen(_attrs,z),_attrs)
      if(batch_idx%3!=0):
        loss_fake = criterion(out_fake,fake_tensor)
      else:
        loss_fake = criterion(out_fake,real_tensor)
        
      loss_fake.backward()
      optimizer_dis.step()
      
      optimizer_gen.zero_grad()
      out_fake2 = nn_Dis(nn_Gen(_attrs,z),_attrs)
      loss_gen = criterion(out_fake2,real_tensor)
      loss_gen.backward()
      optimizer_gen.step()
    print("{}, out_real={},out_fake={},out_fake2={},loss_real={},loss_fake={},loss_gen={}".format(epoch, out_real.mean().item(),out_fake.mean().item(),out_fake2.mean().item(),loss_real.item(),loss_fake.item(),loss_gen.item()))

    gen_z= torch.rand(z_features).unsqueeze(dim=0)
    gen_attr = torch.tensor(images_attr_data[100][1]).unsqueeze(dim=0)
    gen_z=gen_z.to(device)
    gen_attr=gen_attr.to(device)
    #gen_input = torch.cat((gen_attr,gen_z)).unsqueeze(dim=0).to(device)
    gen = nn_Gen(gen_attr,gen_z)
    #gen = gen.squeeze(0)
    vutil.save_image(gen,'./gen/{}.jpg'.format(epoch),normalize=True)
    #img = ToPILImage()(gen.cpu())
    #img.save('./gen/{}.jpg'.format(epoch))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()



0, out_real=0.1810140609741211,out_fake=0.9969666004180908,out_fake2=0.6660587787628174,loss_real=4.203403949737549,loss_fake=5.645343780517578,loss_gen=0.43230777978897095



1, out_real=0.3608998656272888,out_fake=0.8805005550384521,out_fake2=0.020973101258277893,loss_real=2.293501377105713,loss_fake=2.2462544441223145,loss_gen=4.418933391571045



2, out_real=0.3367539346218109,out_fake=0.8803200721740723,out_fake2=0.11369688808917999,loss_real=1.841984748840332,loss_fake=2.1751701831817627,loss_gen=2.3642919063568115



3, out_real=0.5317865014076233,out_fake=0.8633860945701599,out_fake2=0.30340448021888733,loss_real=0.9475818276405334,loss_fake=2.0070979595184326,loss_gen=1.2758290767669678


...


61, out_real=0.532063901424408,out_fake=0.5623260140419006,out_fake2=0.5339232683181763,loss_real=0.6432685852050781,loss_fake=0.8326370716094971,loss_gen=0.6377704739570618



62, out_real=0.5021271705627441,out_fake=0.5527269244194031,out_fake2=0.468858003616333,loss_real=0.7225444912910461,loss_fake=0.8260471820831299,loss_gen=0.7767526507377625



63, out_real=0.5941254496574402,out_fake=0.6444304585456848,out_fake2=0.5360592603683472,loss_real=0.5329583287239075,loss_fake=1.0356662273406982,loss_gen=0.6370053887367249



64, out_real=0.5908634662628174,out_fake=0.6072850227355957,out_fake2=0.4683232009410858,loss_real=0.5406591296195984,loss_fake=0.9428582191467285,loss_gen=0.7810264825820923



65, out_real=0.5526828765869141,out_fake=0.5714403986930847,out_fake2=0.4740493595600128,loss_real=0.6083448529243469,loss_fake=0.8820474743843079,loss_gen=0.8081608414649963


In [ ]:
attributes_test[100][:40]

In [ ]:
z= torch.randn(z_features)
attr = torch.tensor(attributes_test[402])
gen_input = torch.cat((attr,z)).unsqueeze(0).to(device)
x = nn_Gen(gen_input)
x = x.squeeze(0)

In [ ]:
img = ToPILImage()(x.cpu())
img

In [ ]:
filename = "{}{:06d}.jpg".format(image_dir,101)
image = Image.open(filename)
image

In [ ]:
x = transform2(image)
print(x.shape)
img = ToPILImage()(x.cpu())
img

In [ ]:
transform2 = transforms.Compose([transforms.Resize(64),
                                 transforms.CenterCrop(64),
                                transforms.ToTensor()])

In [ ]:
attributes_test[171]

In [ ]:
x = transforms.functional.to_tensor(image)
print(x.shape)
img = ToPILImage()(x.cpu())
img

In [ ]:
#pd.set_option("display.max_columns",None)
#print(list_attr_celeba.head(11))
list_attr_celeba.iloc[100]


References:\
1. https://www.kaggle.com/jessicali9530/celeba-dataset?select=list_attr_celeba.csv \
2. https://machinelearningmastery.com/how-to-implement-wasserstein-loss-for-generative-adversarial-networks/
3. 